In [2]:
import numpy as np
import pandas as pd
import sqlite3

The goal of this notebook is to import US wildfire data from https://www.kaggle.com/rtatman/188-million-us-wildfires and analyze if the wildfires of recent years are normal or abnormal.

Plot the number of wildfires and the total acreage of wildfires over time
Plot for California alone
Plot for entire US

Further Analysis
Plot wildfires on US map

In [3]:
# Import database and load into appropriate dataframes
wf_data = sqlite3.connect("FPA_FOD_20170508.sqlite")
df = pd.read_sql_query("select OBJECTID,FOD_ID,FIRE_NAME,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY from Fires limit 1000", 
                       wf_data) #remove limit for final evaluation

In [4]:
# create separate dataframe for only california
ca_df=df[df['STATE']=='CA']

Two DataFrames created

    df: data from all US fires
    ca_df: data from all California fires
    

In [5]:
year_df = df.groupby('FIRE_YEAR')
year_ca_df = ca_df.groupby('FIRE_YEAR')

In [6]:
year_df.count()['FOD_ID']

FIRE_YEAR
2004     32
2005    968
Name: FOD_ID, dtype: int64

In [7]:
year_ca_df.count()['FOD_ID']

FIRE_YEAR
2004     16
2005    115
Name: FOD_ID, dtype: int64

In [8]:
df

,OBJECTID,FOD_ID,FIRE_NAME,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY
0,1,1,FOUNTAIN,2005,2453403.5,33,1300,0.10,A,40.036944,-121.005833,CA,63
1,2,2,PIGEON,2004,2453137.5,133,0845,0.25,A,38.933056,-120.404444,CA,61
2,3,3,SLACK,2004,2453156.5,152,1921,0.10,A,38.984167,-120.735556,CA,17
3,4,4,DEER,2004,2453184.5,180,1600,0.10,A,38.559167,-119.913333,CA,3
4,5,5,STEVENOT,2004,2453184.5,180,1600,0.10,A,38.559167,-119.933056,CA,3
5,6,6,HIDDEN,2004,2453186.5,182,1800,0.10,A,38.635278,-120.103611,CA,5
6,7,7,FORK,2004,2453187.5,183,1800,0.10,A,38.688333,-120.153333,CA,17
7,8,8,SLATE,2005,2453437.5,67,1300,0.80,B,40.968056,-122.433889,CA,None
8,9,9,SHASTA,2005,2453444.5,74,1200,1.00,B,41.233611,-122.283333,CA,None
9,10,10,TANGLEFOOT,2004,2453187.5,183,1800,0.10,A,38.548333,-120.149167,CA,5
